<a href="https://colab.research.google.com/github/ValdazoAmerico/pytorch-bert-sentiments-analysis/blob/main/bert_pytorch_sentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install transformers

     |████████████████████████████████| 3.1 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 24.6 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
     |████████████████████████████████| 596 kB 49.4 MB/s 
     |████████████████████████████████| 3.3 MB 15.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertModel

In [2]:
data_path = "/content/test.ft.txt"

In [3]:

'''
sentiments = {
  __label__1: 1-2 stars,
  __label__2: 4-5 stars
}
'''

with open(data_path) as f:
  data = f.readlines()

In [4]:

sentiments = []
reviews = []

for review in data[:10000]:
  sentiments.append(review[9])
  reviews.append(review[10:])

In [5]:
len(sentiments), len(reviews)

(10000, 10000)

In [6]:
data[:10]

['__label__2 Great CD: My lovely Pat has one of the GREAT voices of her generation. I have listened to this CD for YEARS and I still LOVE IT. When I\'m in a good mood it makes me feel better. A bad mood just evaporates like sugar in the rain. This CD just oozes LIFE. Vocals are jusat STUUNNING and lyrics just kill. One of life\'s hidden gems. This is a desert isle CD in my book. Why she never made it big is just beyond me. Everytime I play this, no matter black, white, young, old, male, female EVERYBODY says one thing "Who was that singing ?"\n',
 "__label__2 One of the best game music soundtracks - for a game I didn't really play: Despite the fact that I have only played a small portion of the game, the music I heard (plus the connection to Chrono Trigger which was great as well) led me to purchase the soundtrack, and it remains one of my favorite albums. There is an incredible mix of fun, epic, and emotional songs. Those sad and beautiful tracks I especially like, as there's not too 

In [7]:

data_dict = {
    'reviews': reviews,
    'sentiment': sentiments
}

df = pd.DataFrame(data_dict)
df.head()

,reviews,sentiment
0,Great CD: My lovely Pat has one of the GREAT ...,2
1,One of the best game music soundtracks - for ...,2
2,Batteries died within a year ...: I bought th...,1
3,"works fine, but Maha Energy is better: Check ...",2
4,Great for the non-audiophile: Reviewed quite ...,2


In [8]:
df.isnull().sum()

reviews      0
sentiment    0
dtype: int64

In [9]:

mapping = {
    '1': 0,
    '2': 1
}

df['sentiment'] = df['sentiment'].map(mapping)

In [10]:
df['sentiment'].value_counts()

1    5125
0    4875
Name: sentiment, dtype: int64

In [11]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [12]:
encodings = tokenizer.batch_encode_plus(
    df['reviews'].iloc[100],
    add_special_tokens=True,
    max_length=256,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

In [13]:
encodings

{'input_ids': tensor([[ 101,  102,    0,  ...,    0,    0,    0],
        [ 101, 1055,  102,  ...,    0,    0,    0],
        [ 101, 1037,  102,  ...,    0,    0,    0],
        ...,
        [ 101, 1045,  102,  ...,    0,    0,    0],
        [ 101, 1050,  102,  ...,    0,    0,    0],
        [ 101,  102,    0,  ...,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 0,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0]])}

In [14]:
class AmazonReviewsDataset(torch.utils.data.Dataset):
  def __init__(self, df):
    self.df = df
    self.maxlen = 256
    self.tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    review = self.df['reviews'].iloc[index].split()
    review = ' '.join(review)
    sentiment = int(self.df['sentiment'].iloc[index])

    encodings = self.tokenizer.encode_plus(
        review,
        add_special_tokens=True,
        max_length=self.maxlen,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    return {
        'input_ids': encodings.input_ids.flatten(),
        'attention_mask': encodings.attention_mask.flatten(),
        'labels': torch.tensor(sentiment, dtype=torch.long)
    }

In [15]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
len(train_df), len(test_df)

(7000, 3000)

In [16]:
train_dataset = AmazonReviewsDataset(train_df)
valid_dataset = AmazonReviewsDataset(test_df)

In [17]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True
)

valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=32
)

In [18]:
for batch in train_loader:
  print(batch['input_ids'].shape)
  print(batch['attention_mask'].shape)
  print(batch['labels'].shape)
  break

torch.Size([32, 256])
torch.Size([32, 256])
torch.Size([32])


In [19]:

class SentimentClassifier(nn.Module):
  def __init__(self):
    super(SentimentClassifier, self).__init__()
    self.distilbert = DistilBertModel.from_pretrained("distilbert-base-uncased")
    self.drop0 = nn.Dropout(0.25)
    self.linear1 = nn.Linear(3072, 512)
    self.relu1 = nn.ReLU()
    self.drop1 = nn.Dropout(0.25)
    self.linear2 = nn.Linear(512, 2)
    self.relu2 = nn.ReLU()

  def forward(self, input_ids, attention_mask):
    outputs = self.distilbert(input_ids, attention_mask)
    last_hidden_state = outputs[0]
    pooled_output = torch.cat(tuple([last_hidden_state[:, i] for i in [-4, -3, -2, -1]]), dim=-1)
    x = self.drop0(pooled_output)
    x = self.relu1(self.linear1(x))
    x = self.drop1(x)
    x = self.relu2(self.linear2(x))
    return x

In [20]:
model = SentimentClassifier()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


SentimentClassifier(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Lin

In [21]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
epochs = 5

In [22]:
for epoch in range(epochs):
  # TRAIN
  model.train()
  train_loop = tqdm(train_loader)
  for batch in train_loop:
    optimizer.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    output = model(input_ids, attention_mask)
    loss = criterion(output, labels)
    loss.backward()
    nn.utils.clip_grad_norm(model.parameters(), max_norm=1.0)
    optimizer.step()

    train_loop.set_description(f"Training Epoch: {epoch}")
    train_loop.set_postfix(loss=loss.item())

  # VALIDATION
  model.eval()
  valid_loop = tqdm(valid_loader)
  for batch in valid_loop:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    output = model(input_ids, attention_mask)
    loss = criterion(output, labels)

    valid_loop.set_description(f"Validation Epoch: {epoch}")
    valid_loop.set_postfix(loss=loss.item())

  0%|          | 0/219 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  del sys.path[0]


  0%|          | 0/94 [00:00<?, ?it/s]

  0%|          | 0/219 [00:00<?, ?it/s]

  0%|          | 0/94 [00:00<?, ?it/s]

...

  0%|          | 0/94 [00:00<?, ?it/s]

  0%|          | 0/219 [00:00<?, ?it/s]

  0%|          | 0/94 [00:00<?, ?it/s]

  0%|          | 0/219 [00:00<?, ?it/s]

  0%|          | 0/94 [00:00<?, ?it/s]

In [25]:
test_sample = test_df['reviews'].iloc[102]
original_label = test_df['sentiment'].iloc[102]

print(test_sample)
print(original_label)

encodings = tokenizer.encode_plus(
    test_sample,
    add_special_tokens=True,
    max_length=256,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

with torch.no_grad():
  model.to('cpu')
  preds = model(encodings['input_ids'].to('cpu'), encodings['attention_mask'].to('cpu'))
  preds = np.argmax(preds)
  output = preds.item()
  print(output+1)

 The First Iron lady?: This is an excellent book and a lively and interesting read. Anne Somerset gives an insight into all sides of the character of Elizabeth. Her moods and feelings, (she was renowned at the time for her short temper), both as a woman and a Queen.The glamour of the Tudor court and what it was like to be in or out of favour in a clique where a careless word could in some cases cost you your life.Elizabeth did not suffer fools gladly and the book examines her religious feelings, her sexuality and the way she dealt with conflicts both at home and abroad in what was essentially a man's world. Elizabeth was nobody's fool and although she would listen to her advisers, she had a mind of her own, shrewd and extremely intelligent, and was quite capable of using her authority as monarch to make important decisions for herself.The book is a delight for anyone with the slightest interest in history.

1
1
